# Training a CNN 

Make a copy of './configs/default.yml' and edit parameters as desired. Point to your config file in second cell.

Run this notebook to train a Convolutional Neural Network using OpenSoundscape's CNN class. For documentation and tutorials, visit [opensoundscape.org](https://opensoundscape.org). 

Import packages:

In [50]:
import opensoundscape as opso 
from load_cfg import cnn_from_cfg
import yaml
import pandas as pd
import wandb
from pathlib import Path
from glob import glob

Load your config file: change the first line to the location of your config file.

In [37]:
config_file = "./configs/default.yml"

with open(config_file, "r") as f:
    cfg = yaml.safe_load(f)

Generate an opensoundscape.CNN object based on the config file parameters

In [15]:
cnn = cnn_from_cfg(config_file)

Load table of annotated clips

In [16]:
df = pd.read_csv('/Users/SML161/labeled_datasets/rana_sierrae_2022/labels_2s.csv')

# point to the location of the audio files
data_root = Path('/Users/SML161/labeled_datasets/rana_sierrae_2022/mp3')
df['file']=[data_root/f for f in df['file']]

# set indices according to OpenSoundscape's expected format
df=df.set_index(['file','start_time','end_time'])

#subset table to desired classes (columns)
df=df[cfg['class_list']]

Split labeled data into training and validation sets

In [22]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df,test_size=0.2,random_state=0)
val_df.to_csv('./validation_set.csv')

Inspect class imbalance

In [49]:
train_df.sum()

A    2136
B      88
C     404
D     288
E     584
dtype: int64

Resample training set for even representation of each class, with 300 of each call type

In [51]:
balanced_train = opso.data_selection.resample(train_df,300,upsample=True,downsample=True)

Train the CNN according to the parameters in the config file. 

Saves model checkpoints and the config to the folder specified in `cfg["train"]["save_path"]` 

In [41]:
#Set up weights and biases logging, and a subfolder for saved checkpoints & config of this run

save_dir = Path(cfg["train"]["save_path"])
if cfg["wandb_init"]["project"] is not None:
    wandb.login()  # only needed once per machine (find API key at wandb.ai/authorize)
    wandb_session = wandb.init(**cfg["wandb_init"])
    run_name = wandb_session.name
else:
    wandb_session = None
    run_number = len(glob(f"{save_dir}/run_*"))+1
    run_name = f"run_{run_number}"

# define and create sub-folder for this run's checkpoints
run_dir = save_dir / run_name
run_dir.mkdir(exist_ok=False)

# update the save path to the newly created subfolder
cfg["train"]["save_path"] = str(run_dir)

# save the config file to the subfolder as well
with open(run_dir / "config.yml",'w') as f:
    yaml.safe_dump(cfg,f)

# train CNN using several parameters defined in the config
cnn.train(
    train_df=balanced_train,
    validation_df=val_df,
    wandb_session=wandb_session,
    **cfg["train"]
)
 
wandb.finish() # notifies wandb that the run completed successfully


Training Epoch 0
Epoch: 0 [batch 0/143, 0.00%] 
	DistLoss: 34.335
Metrics:
Epoch: 0 [batch 10/143, 6.99%] 
	DistLoss: 17.449


/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in labels with no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


Metrics:
Epoch: 0 [batch 20/143, 13.99%] 
	DistLoss: 16.614
Metrics:
Epoch: 0 [batch 30/143, 20.98%] 
	DistLoss: 15.297


/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in labels with no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


Metrics:
Epoch: 0 [batch 40/143, 27.97%] 
	DistLoss: 14.507


...

Metrics:
Epoch: 5 [batch 90/143, 62.94%] 
	DistLoss: 11.639
Metrics:
Epoch: 5 [batch 100/143, 69.93%] 
	DistLoss: 11.633
Metrics:
Epoch: 5 [batch 110/143, 76.92%] 
	DistLoss: 11.569
Metrics:
Epoch: 5 [batch 120/143, 83.92%] 
	DistLoss: 11.585
Metrics:
Epoch: 5 [batch 130/143, 90.91%] 
	DistLoss: 11.570
Metrics:
Epoch: 5 [batch 140/143, 97.90%] 
	DistLoss: 11.610
Metrics:


/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/var/folders/d8/265wdp1n0bn_r85dh3pp95fh0000gq/T/tmpe7ki17n0wandb'>
  _warnings.warn(warn_message, ResourceWarning)
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/var/folders/d8/265wdp1n0bn_r85dh3pp95fh0000gq/T/tmp_vvrcdyywandb'>
  _warnings.warn(warn_message, ResourceWarning)
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/var/folders/d8/265wdp1n0bn_r85dh3pp95fh0000gq/T/tmp8lbaijr8wandb'>
  _warnings.warn(warn_message, ResourceWarning)
/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/tempfile.py:821: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/var/folders/d8/265wdp1n0bn_r85dh3pp95fh0000gq/T/tmpx7nnhkbrwandb'>
  _warnings.warn(warn_message, ResourceWarning)
/Users/SML16

Metrics:
	MAP: 0.078

Validation.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/torch/__init__.py", line 1239, in <module>
    from torch import onnx as onnx
  File "/Users/SML161/miniconda3/envs/opso_dev/lib/python3.9/site-packages/torch/onnx/__init__.py", line 12, in <module>
    from . import (  # usort:skip. Keep the order instead of sorting lexicographically
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_externa

KeyboardInterrupt: 

Save a model checkpoint

In [46]:
cnn.save_weights(run_dir / 'latest.pt')